# INTRODUCTION

In this exercise, you will assist a company that sells IT accessories in answering some business questions using data and visuals, thereby contributing to enhancing their decision-making processes quickly and effectively.

Explore the provided invoices data at each sales point alongside other company's files and answer the following questions. **You are encouraged to use only the method/approaches discussed in our class where possible.**

# With a help of an appropriate graph, Illustrate and comment on the distribution of total products sold at each sales point?

In [11]:
# Your Answer
import pandas as pd


final_products = pd.read_csv('final_products.csv')
final_invoices_downtown = pd.read_csv('final_invoices_downtown.csv')
final_invoices_commercial = pd.read_csv('final_invoices_commercial.csv')
final_invoice_lines = pd.read_csv('final_invoice_lines.csv')
final_customers = pd.read_csv('final_customers.csv')
final_employees = pd.read_csv('final_employees.csv')
final_products['product_category']


0           Storage
1    InputAndOutput
2           Storage
3          Wireless
4            Others
5          Wireless
6      Input/Output
7          Wireless
8           Storage
9          Wireless
Name: product_category, dtype: object

In [12]:
from collections import defaultdict
import plotly.graph_objects as go
product_mapping = {number: name for number, name in zip(final_products['product_number'], final_products['product_name'])}
product_quantities = defaultdict(int)
for code, quantity in zip(final_invoice_lines['product_code'], final_invoice_lines['quantity']):
    product_quantities[product_mapping[code]] += quantity
fig = go.Figure(data=[go.Bar(x=list(product_quantities.keys()), y=list(product_quantities.values()), marker=dict(color='skyblue'))])
fig.update_layout(
    xaxis_title='Product Name',
    yaxis_title='Total Quantity Sold',
    title='Distribution of Total Quantity Sold for Each Product',
    xaxis_tickangle=-45,
)
fig.show()



#### Distribution of Total Products Sold at Each Sales Point
- This barchart shows the distribution of total products sold at each sales point (total quantity sold)

# Effectively display the revenue generated by each sales officer over time. Explain any quick insights from the resulting graph.

In [13]:
def retrieve_emp_id(employee_number):
    return int(employee_number[4:]) 

final_employees['sales_off_id'] = final_employees['employee_number'].apply(retrieve_emp_id)
final_employees['sales_off_id']


0    1
1    2
2    3
3    4
4    5
5    6
6    7
Name: sales_off_id, dtype: int64

In [14]:
final_products['product_profit'] = final_products['selling_price'] - final_products['purchase_price']
final_products

,product_number,product_category,product_name,purchase_price,selling_price,product_profit
0,1,Storage,External Hard Drive,99,110,11
1,2,InputAndOutput,Portable Bluetooth Speaker,55,60,5
2,3,Storage,Power Bank,25,40,15
3,4,Wireless,Wireless Mouse,6,10,4
4,5,Others,Laptop Stand,45,50,5
5,6,Wireless,Wireless Charging Pad,60,70,10
6,7,Input/Output,WebCam,15,18,3
7,8,Wireless,WiFi Range Extender,39,45,6
8,9,Storage,Flash Drive,8,15,7
9,10,Wireless,Wireless Keyboard,27,30,3


In [15]:

final_invoice_lines
merged_data = pd.merge(final_invoice_lines, final_products, left_on='product_code', right_on='product_number', how='left')
merged_data['invoice_line_profit'] = merged_data['quantity'] * merged_data['product_profit']

merged_data

,invoice_id,product_code,quantity,product_number,product_category,product_name,purchase_price,selling_price,product_profit,invoice_line_profit
0,1,7,1,7,Input/Output,WebCam,15,18,3,3
1,1,8,1,8,Wireless,WiFi Range Extender,39,45,6,6
2,1,9,3,9,Storage,Flash Drive,8,15,7,21
3,1,5,1,5,Others,Laptop Stand,45,50,5,5
4,1,10,3,10,Wireless,Wireless Keyboard,27,30,3,9
...,...,...,...,...,...,...,...,...,...,...
2663,613,4,1,4,Wireless,Wireless Mouse,6,10,4,4
2664,613,2,1,2,InputAndOutput,Portable Bluetooth Speaker,55,60,5,5
2665,613,9,4,9,Storage,Flash Drive,8,15,7,28
2666,613,8,3,8,Wireless,WiFi Range Extender,39,45,6,18


In [16]:
combined_invoices = pd.concat([final_invoices_downtown, final_invoices_commercial], ignore_index=True)

merged_data['invoice_line_profit'] = (merged_data['selling_price'] - merged_data['purchase_price']) * merged_data['quantity']

invoice_profit = merged_data.groupby('invoice_id')['invoice_line_profit'].sum().reset_index()

combined_invoices = pd.merge(combined_invoices, invoice_profit, on='invoice_id', how='left')
combined_invoices['invoice_profit'] = invoice_profit['invoice_line_profit']

billing_phone_number = combined_invoices['billing_phone']

served_customers = pd.merge(final_customers, combined_invoices, left_on='phone_number', right_on='billing_phone', how='inner')
served_customers = pd.merge(final_employees,served_customers,  left_on='sales_off_id', right_on='sales_person_id', how='left')

served_customers


,employee_number,last_name_x,first_name_x,line_manager,title,sales_off_id,first_name_y,last_name_y,country,phone_number,sales_person_id,invoice_id,invoice_date,billing_phone,invoice_line_profit,invoice_profit
0,emp_1,Ross,Shannon,NaN,General Manager,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,emp_2,Griffin,Mark,1.0,Sales Manager,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,emp_03,Estrada,Patricia,2.0,Sales Officer,3,Anita,Ruiz,Canada,2.507667e+11,3.0,47.0,2021-04-02,2.507667e+11,73.0,101.0
3,emp_03,Estrada,Patricia,2.0,Sales Officer,3,Anita,Ruiz,Canada,2.507667e+11,3.0,68.0,2020-12-17,2.507667e+11,101.0,95.0
4,emp_03,Estrada,Patricia,2.0,Sales Officer,3,Anita,Ruiz,Canada,2.507667e+11,3.0,157.0,2023-06-03,2.507667e+11,12.0,92.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,emp_05,Petersen,Cassidy,2.0,Sales Officer,5,Sara,Lee,USA,2.507667e+11,5.0,397.0,2022-12-06,2.507667e+11,113.0,77.0
518,emp_05,Petersen,Cassidy,2.0,Sales Officer,5,Sara,Lee,USA,2.507667e+11,5.0,400.0,2023-07-29,2.507667e+11,72.0,38.0
519,emp_05,Petersen,Cassidy,2.0,Sales Officer,5,Sara,Lee,USA,2.507667e+11,5.0,500.0,2023-01-08,2.507667e+11,114.0,15.0
520,emp_6,Gomez,Charles,1.0,IT Manager,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:

sales_manager_profit = served_customers.groupby('sales_person_id')['invoice_profit'].sum().reset_index()

sales_manager_profit = pd.merge(sales_manager_profit, final_employees, left_on='sales_person_id', right_on='sales_off_id', how='left')
sales_manager_profit


sales_manager_profit['sales_manager_full_name'] = sales_manager_profit['first_name'] + ' ' + sales_manager_profit['last_name']

sales_manager_profit.rename(columns={'invoice_profit': 'profit'}, inplace=True)
sales_manager_profit.drop(columns=['sales_off_id', 'first_name', 'last_name'], inplace=True)
sales_manager_profit



,sales_person_id,profit,employee_number,line_manager,title,sales_manager_full_name
0,3.0,20888.0,emp_03,2.0,Sales Officer,Patricia Estrada
1,4.0,8733.0,emp_04,2.0,Sales Officer,Alicia West
2,5.0,15440.0,emp_05,2.0,Sales Officer,Cassidy Petersen


In [21]:
import pandas as pd
import plotly.express as px

pie_data = {
    'sales_manager_full_name':sales_manager_profit['sales_manager_full_name'] ,
    'total_revenue': sales_manager_profit['profit']
}

sales_manager_revenue_mock = pd.DataFrame(pie_data)

fig = px.pie(sales_manager_revenue_mock, values='total_revenue', names='sales_manager_full_name', title='Revenue generated by Sales officesr')
fig.show()



# Determine the most active days of the week for each product category by analyzing the average number of products sold and display the result using an appropriate graph.  What are your thoughts on the resulting graph?

In [19]:
from datetime import datetime

def get_weekday_name(date_string):
    try:
        date_object = datetime.strptime(date_string, "%Y-%m-%d")
        weekday_name = date_object.strftime("%A")
        return weekday_name
    except ValueError:
        return "Invalid date format. Please use YYYY-MM-DD format."
    
    
merged_data
combined_invoices
merged_dataset = pd.merge(merged_data, combined_invoices, on='invoice_id', how='inner')
merged_dataset['week_day_name'] = merged_dataset['invoice_date'].apply(get_weekday_name)

merged_dataset

,invoice_id,product_code,quantity,product_number,product_category,product_name,purchase_price,selling_price,product_profit,invoice_line_profit_x,invoice_date,billing_phone,invoice_line_profit_y,invoice_profit,week_day_name
0,1,7,1,7,Input/Output,WebCam,15,18,3,3,2020-12-14,250766666618,88,99,Monday
1,1,8,1,8,Wireless,WiFi Range Extender,39,45,6,6,2020-12-14,250766666618,88,99,Monday
2,1,9,3,9,Storage,Flash Drive,8,15,7,21,2020-12-14,250766666618,88,99,Monday
3,1,5,1,5,Others,Laptop Stand,45,50,5,5,2020-12-14,250766666618,88,99,Monday
4,1,10,3,10,Wireless,Wireless Keyboard,27,30,3,9,2020-12-14,250766666618,88,99,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2663,613,4,1,4,Wireless,Wireless Mouse,6,10,4,4,2021-02-27,250766666620,85,59,Saturday
2664,613,2,1,2,InputAndOutput,Portable Bluetooth Speaker,55,60,5,5,2021-02-27,250766666620,85,59,Saturday
2665,613,9,4,9,Storage,Flash Drive,8,15,7,28,2021-02-27,250766666620,85,59,Saturday
2666,613,8,3,8,Wireless,WiFi Range Extender,39,45,6,18,2021-02-27,250766666620,85,59,Saturday


In [20]:
grouped_data = merged_dataset.groupby('week_day_name')['quantity'].sum().reset_index()

traces = go.Bar(x=grouped_data['week_day_name'], y=grouped_data['quantity'])

layout = go.Layout(
    title='Total Number of Products Sold by Day of the Week',
    xaxis=dict(title='Day of the Week'),
    yaxis=dict(title='Total Products Sold'),
)

fig = go.Figure(data=traces, layout=layout)
fig.show()

# Reshape the data to display the total revenue amount of each product by year side by side. Dispaly the resulting dataframe.

In [30]:
def extract_year(date_string):
    date_obj = datetime.strptime(date_string, '%Y-%m-%d')
    year = date_obj.year
    return year

merged_dataset['year'] = merged_dataset['invoice_date'].apply(extract_year)

pivot_table = merged_dataset.pivot_table(index='product_name', columns='year', values='product_profit', aggfunc='sum')
pivot_table

year,2020,2021,2022,2023,2024
product_name,,,,,
External Hard Drive,561,627,726,638,220
Flash Drive,364,441,546,385,119
Laptop Stand,290,275,365,350,75
Portable Bluetooth Speaker,285,350,310,290,100
Power Bank,750,840,1215,840,285
WebCam,171,189,228,186,60
WiFi Range Extender,360,348,438,384,150
Wireless Charging Pad,600,570,710,570,170
Wireless Keyboard,180,171,204,174,54
